# Export as a Relion Project
This tutorial covers how one can export the generate simulations in previous steps so it can be imported
into common cryo-EM processing frameworks, such as RELION, cryoSPARC, and others. The export_particles
method uses an Export class that receives the particles metadata and performs the following:
- Create a structured RELION project folder
- Format the simulated particle metadata
- Mimic the RELION Extract Particles job that produces the "particles.star" file
- Copy/Move/Symlink the current simulated images into this project folder

## Initialize Libraries

In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd()) + "/../src")

import EMCrafter
from EMCrafter.sim import SimResults
EMCrafter.base._set_logging(minimal=True)

## Load simulated dataset

In [2]:
# From previous tutorials, we can load the simulated dataset with:
sim = SimResults().load("data/pickle/dataset_full.pkl")

# If you had many separate simulations, you can create a list with their paths:
# sim_paths = ["sim1_path", "sim2_path", ..., "simN_path"]
# And merge them with a simple:
# sim = SimResults().merge(sim_paths)

Loaded SimResults object from data/pickle/dataset_full.pkl


## Export

In [3]:
# Output folder containing the relion project structure:
export_output = "data/rln_project"  

# Export type:
export_type   = "link"
# Export type needs to be one of ["copy", "move", "link"]
# - copy: copy the particle files into this relion project
# - move: move the particle files into this relion project (the simulation will not be able to read those anymore)
# - link: uses symlink so it does not need to copy/move anything (recomended)

# Exporting:
df = sim.export_particles(
    export_output,          # Output folder
    export_type,            # Type of file building
    force=True)             # If the folder already exists, you need force=True for overriding
# The output will be a pandas DataFrame, related to the star file built.

Starting 'export_particles'...
Exporter initialized
Output directory exists, existing files may be overwritten.
Output directory set to: data/rln_project.
> Export: Validate arguments
> Export: Format optics
> Export: Format particles
> Export: Build physical files
> Export: Compile STAR file
Finished 'export_particles' in 0.14s.


In [4]:
df.head(5)

,rlnImageName,rlnMicrographName,rlnOpticsGroup,rlnClassNumber,rlnCoordinateX,rlnCoordinateY,rlnDefocusU,rlnDefocusV,rlnDefocusAngle,rlnCtfBfactor,rlnCtfScalefactor,rlnPhaseShift,rlnAutopickFigureOfMerit,rlnCtfMaxResolution,rlnCtfFigureOfMerit
0,1@Extract/job001/Movies/micrograph_00000_noisy...,Movies/micrograph_00000_noisy_0.mrc,1,0,1926,7853,9336.855410,9336.855410,0.0,0.0,1.0,0.0,0.15,5.0,0.05
1,2@Extract/job001/Movies/micrograph_00000_noisy...,Movies/micrograph_00000_noisy_0.mrc,1,0,4595,1946,12036.420574,12036.420574,0.0,0.0,1.0,0.0,0.15,5.0,0.05
2,3@Extract/job001/Movies/micrograph_00000_noisy...,Movies/micrograph_00000_noisy_0.mrc,1,0,9776,3011,12128.981862,12128.981862,0.0,0.0,1.0,0.0,0.15,5.0,0.05
3,4@Extract/job001/Movies/micrograph_00000_noisy...,Movies/micrograph_00000_noisy_0.mrc,1,0,6021,2773,15117.474892,15117.474892,0.0,0.0,1.0,0.0,0.15,5.0,0.05
4,5@Extract/job001/Movies/micrograph_00000_noisy...,Movies/micrograph_00000_noisy_0.mrc,1,0,2779,2543,11527.072740,11527.072740,0.0,0.0,1.0,0.0,0.15,5.0,0.05


## Import to cryoSPARC

#### 1. Build job
To import the extracted file to cryoSPARC, open your workspace, click in builder and search for "Import Particle Stack"

<img src="images/cs_01_importJob.png" width="500">

#### 2. File setup
Once the job is created, build it by setting:
- Particle meta path: Metadata file, which is the one located in "rln_project/Extract/job001/particles.star"
- Particle data path: The base folder of the project, so simply "rln_project"

<img src="images/cs_02_importPaths.png" width="300">

#### 3. Experimental setup (optional)
You can override experimental setup, if needed. This should have already been correctly setup
on the exported files if you have followed the tutorial throughly. In any case, if needed:

<img src="images/cs_03_importSetup.png" width="300">

#### 4. Strict Checking (optional)
To make sure everything is working with the files, enable the strict checking, using:
- Particle raw data (blob)
- Particle CTF parameters (ctf)
- Particle pick statistics (pick_stats)


<img src="images/cs_04_importChecks.png" width="280">

#### 5. Queue
Then queue the import job. Once it is finished, you should see something like this:

<img src="images/cs_05_importResult.png" width="500">

So you may start other jobs using this imported particle stack.

## Import to RELION

#### 1. Initialize Project
For RELION:
- Open your terminal
- cd into the "rln_project" directory
- Start relion

<img src="images/rln_01_open.png" width="200">

#### 2. Go on from there
Once you start the relion package, it should already identify the (fake) particle extraction job:

<img src="images/rln_02_initialized.png" width="200">

So you may start other jobs simply by selecting the STAR particles file located at "rln_project/Extract/job_0001/particles.star

<img src="images/rln_03_newJobs.png" width="500">